# **実務的なデータエクスポートとエリア選定**
個人事業として成果を納品する際、GEEのCode Editorの画面だけでなく、お客様が利用できる形式（CSV、GeoTIFF、GISファイル）でデータを提供する必要があります。

この課題では、特定した**緑化ポテンシャルエリア**の中から、クライアント（京都市役所や建設会社）に**具体的な対策地リスト**を提示するための最終ステップを行います。

## **課題 5: 対策地リストの作成とデータ納品準備**

### **ステップ1: 緑化ポテンシャルエリアのベクトル化（フィーチャ変換)**
1. 対策地画像（$greeningPotential\_2023$）をポリゴンフィーチャに変換します。これにより、GISソフトで開ける地理情報データになります。$greeningPotential\_2023$ 画像に対して $ee.Image.reduceToVectors()$ 関数を適用し、ポリゴンフィーチャコレクションに変換します。
2. AOI（$kyoto\_aoi$）でクリップし、スムーズなポリゴンにするために適切な**連結性（8）**を指定します。

**【ヒント】**

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='earth-change-analysis')

In [ ]:
kyoto_aoi = ee.Geometry.Rectangle(135.29, 34.80, 136.30, 35.50)

l8_base_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(kyoto_aoi)\
    .filterDate('2013-07-01', '2023-08-31')\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

l8_2013 = l8_base_collection.filterDate('2013-07-01', '2013-08-31')
l8_2023 = l8_base_collection.filterDate('2023-07-01', '2023-08-31')


In [ ]:
image_2013 = l8_2013.median()
image_2023 = l8_2023.median()

def calc_LST(image, year):
    return image.select('ST_B10')\
        .multiply(0.00341802).add(149.0).subtract(273.15)\
        .rename(f'{year}_LST_Celsius')

LST_2013 = calc_LST(image_2013, '2013')
LST_2023 = calc_LST(image_2023, '2023')

LST_diff = LST_2023.subtract(LST_2013).rename('LST_Diff')

landcover = ee.ImageCollection("ESA/WorldCover/v200")
landcover_median = landcover.median()

builtup_mask = landcover_median.eq(50)

builtup_LST_2023 = LST_2023.updateMask(builtup_mask)

hotspot_2023 = builtup_LST_2023.gt(37.5)

nirBand = 'SR_B5'
redBand = 'SR_B4'

NDVI_2023 = image_2023.normalizedDifference([nirBand, redBand]).rename('NDVI')

NDVI_low_2023 = NDVI_2023.lt(0.25)

greeningPotential_2023 = hotspot_2023.And(NDVI_low_2023)

In [ ]:
# JavaScriptの true → Pythonでは True（大文字！）
vectors = greeningPotential_2023.reduceToVectors(
    geometry=kyoto_aoi,
    scale=10,
    maxPixels=1e12,
    eightConnected=True    # 8近傍. 上下左右＋斜めの8方向すべてを「隣接」とみなす
).map(
    lambda feature: feature.set('area_sq_m', feature.geometry().area(1))  # lambda:1行で書ける関数を作る
)

---

|問題	                   |解決策                                        |
|--------------------------|---------------------------------------------|
|geometry().area() でエラー |geometry().area(1) のように 誤差許容値を指定する  |

---

### **ステップ 2: 優先度の高い対策地の抽出（面積フィルタリング）**
対策地のポリゴンは非常に細かくなるため、実務的な対策を行うには**ある程度の大きさ**が必要です。

1. ステップ1で作成したフィーチャコレクションから、**面積が10,000 $m^2$（1ヘクタール）以上**のポリゴンのみをフィルタリングします。
2. フィルタリングされたフィーチャコレクションを地図に追加し、最も優先すべき対策地がどこにあるかを確認します。

In [ ]:
filtered_vectors = vectors.filter(ee.Filter.gt('area_sq_m', 10000))

In [ ]:
map = geemap.Map()
map.add_layer(
    filtered_vectors,
    {
        'color': 'orange',
        'fillColor': 'orange',
        'width': 1
    },
    'Green Potential Polygons'
)
map.centerObject(filtered_vectors, 12)
map                                     

---
|関数	                        |引数  	          |用途                 |
|-------------------------------|-----------------|--------------------|
|set_center(lon, lat, zoom)  	|数値	          |手動で地図の中心を指定  |
|centerObject(ee_object, zoom)  |GEEオブジェクト	  |自動でその範囲にズーム  |
---

### **ステップ 3: データのエクスポート**

最終的な成果物として、以下の2種類のエクスポートタスクを設定します。

1. **フィーチャ（対策地ポリゴン）のエクスポート**:
- 面積が $10,000 \text{ m}^2$ 以上のフィーチャコレクションを、クライアントが利用しやすい**CSVファイル**としてGoogle Driveにエクスポートするタスクを設定します。
   - （ヒント: `Export.table.toDrive` を使用）
   
2. **ラスター（LST差分）のエクスポート**:
- 課題3で作成したLST差分画像（2023年中央値 - 2013年中央値）を、クライアントが利用しやすい**GeoTIFFファイル**としてGoogle Driveにエクスポートするタスクを設定します。（
  - ヒント: `Export.image.toDrive` を使用）

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=filtered_vectors,
    description='green_potential_polygons',
    folder='GEE_1000_Challenge',
    fileFormat='CSV'
)
task.start()

In [ ]:
def calc_LST(image, year):
    return image.select('ST_B10')\
        .multiply(0.00341802).add(149.0).subtract(273.15)\
        .rename(f'{year}_LST_Celsius')

LST_2013 = calc_LST(image_2013, '2013')
LST_2023 = calc_LST(image_2023, '2023')

LST_diff = LST_2023.subtract(LST_2013).rename('LST_Diff')

In [ ]:
print(kyoto_aoi.getInfo()['type'])

In [ ]:
task = ee.batch.Export.image.toDrive(
    image=LST_diff,
    description='LST_diff_2013_2023',
    folder='GEE_1000_Challenge',
    region=kyoto_aoi,
    scale=10,
    fileFormat='GeoTIFF'
)
task.start()

In [ ]:
task.status()

# これでヒートアイランド解析から具体的な対策地選定、そして**実務的なデータ納品（エクスポート）**まで、一連のプロジェクトワークフローを完璧に完了しました。 
---

特に、以下の重要なスキルが確認できました。
1. **ラスターからベクトルへの変換（`reduceToVectors`）:** 対策地の画像をGISソフトウェアで利用可能なポリゴンデータに変換し、地理空間データ処理の幅が広がりました。
2. **フィーチャプロパティの追加（`.map(lambda ...`)）:** ベクトルに面積情報を追加することで、データの属性情報が充実しました。
3. **属性によるフィルタリング（ee.Filter.gt）:** 面積が $10,000 \text{ m}^2$ 以上の、**実務的に意味のある大規模な対策地**のみを抽出する高度なデータ選別を行いました。
4. **データのエクスポート（`Export.table.toDrive` / `Export.image.toDrive`）:** ラスターデータ（LST差分）とベクターデータ（対策地ポリゴン）の両方を、標準的なファイル形式（GeoTIFF、CSV）でGoogle Driveにエクスポートする設定を完了しました。

  
これで、お客様に**「このGeoTIFFでLSTの変化を確認できます」、「このCSV/GeoJSONで、優先すべき $1 \text{ ha}$ 以上の緑化対策地リストを提供します」**と提案できる、具体的な成果物が出揃いました。
---